In [24]:
# debug_and_fix_tags.py
import zmq, msgpack, msgpack_numpy as m
m.patch()
import numpy as np, time, math

PORT = 5557
TAG4_ID = 4
TAG4_HEIGHT_M = 0.075  # same as your system
UP = np.array([0.0, 0.0, 1.0])

ctx = zmq.Context()
sock = ctx.socket(zmq.SUB)
sock.setsockopt(zmq.SUBSCRIBE, b"")
sock.connect(f"tcp://127.0.0.1:{PORT}")

def rot_err(R):
    I = np.eye(3)
    return np.linalg.norm(R.dot(R.T) - I), np.linalg.det(R)

def vec_info(name, v):
    return f"{name}: [{v[0]:+.3f}, {v[1]:+.3f}, {v[2]:+.3f}] | |v|={np.linalg.norm(v):.3f}"

def align_local_z_to_world_up(T):
    """
    Returns T_fixed = T.dot(R_local) where R_local rotates tag-local axes
    so that tag-local +Z (0,0,1) maps to world UP.
    This preserves tag position but changes its orientation so its Z aligns to UP.
    """
    R = T[:3,:3]
    n = R[:,2]  # current tag Z in world
    n = n / np.linalg.norm(n)
    # target k in tag-local coordinates: we want R * (R_local * e_z) = UP
    # So R_local * e_z = R.T @ UP = k_local
    k_local = R.T.dot(UP)
    k_local = k_local / np.linalg.norm(k_local)
    # now compute rotation that maps e_z -> k_local in local coordinates
    ez = np.array([0.0, 0.0, 1.0])
    v = np.cross(ez, k_local)
    s = np.linalg.norm(v)
    c = np.dot(ez, k_local)
    if s < 1e-8:
        # already aligned or opposite
        if c > 0:
            R_local = np.eye(3)
        else:
            # 180 deg rotation about any orth axis (choose x)
            R_local = np.array([[1,0,0],[0,-1,0],[0,0,-1]], float)
    else:
        vx = np.array([[0, -v[2], v[1]],[v[2], 0, -v[0]],[-v[1], v[0], 0]], float)
        R_local = np.eye(3) + vx + vx.dot(vx) * ((1 - c) / (s**2))
    T_fixed = np.eye(4)
    T_fixed[:3,:3] = R.dot(R_local)  # new world rotation
    T_fixed[:3,3] = T[:3,3].copy()
    return T_fixed

print("[debug] waiting for snapshot...")
msg = sock.recv()   # blocking for one snapshot
data = msgpack.unpackb(msg, object_hook=m.decode, strict_map_key=False)

tags = data.get("tags", {})
cams = data.get("cameras", {})

print("--- SNAPSHOT CHECK ---\n")
for tid, T in tags.items():
    T = np.array(T, dtype=float)
    pos = T[:3,3]
    R = T[:3,:3]
    err, det = rot_err(R)
    n = R[:,2]  # tag local +Z in world
    print(f"TAG {tid}")
    print(f"  pos = {pos}")
    print(f"  normal (world) = {n}  | dot(up) = {n.dot(UP):.3f}")
    print(f"  rot ortho err = {err:.6f}, det = {det:.6f}")
    # check if tag is floor-like (z small) or mobile
    z = pos[2]
    print(f"  z = {z:.4f} m")
    # decide if orientation is bad for a floor tag
    if tid != TAG4_ID:
        if abs(z) < 0.5:
            # expected floor tag -> normal dot up should be near ±1
            if abs(n.dot(UP)) < 0.8:
                print("  >>> WARNING: tag normal not aligned with UP (likely tilt/vertical).")
                T_fix = align_local_z_to_world_up(T)
                nfix = T_fix[:3,:3][:,2]
                print("    -> Suggest fixed normal (world) = ", nfix, "pos unchanged:", T_fix[:3,3])
            else:
                print("  orientation looks OK for floor tag.")
        else:
            print("  tag has high z; may be on raised surface (bed/shelf).")
    else:
        # mobile tag: check height near TAG4_HEIGHT_M
        if abs(z - TAG4_HEIGHT_M) > 0.05:
            print(f"  >>> WARNING: mobile tag z != {TAG4_HEIGHT_M:.3f} m (delta {z-TAG4_HEIGHT_M:+.3f})")
            # give fix that sets z to expected height but keeps orientation
            T_fixed = T.copy()
            T_fixed[2,3] = TAG4_HEIGHT_M
            print("    -> suggested T with corrected height (translation):", T_fixed[:3,3])
        # mobile orientation check (often vertical or horizontal depending on mount)
        print("  mobile tag normal dot(up) = ", n.dot(UP))

print("\nCameras:")
for cname, T in cams.items():
    T = np.array(T, dtype=float)
    pos = T[:3,3]
    R = T[:3,:3]
    err, det = rot_err(R)
    print(f"CAM {cname}: pos={pos}, rot_err={err:.6f}, det={det:.6f}")

print("\n--- DONE ---")


[debug] waiting for snapshot...
--- SNAPSHOT CHECK ---

TAG 1
  pos = [0. 0. 0.]
  normal (world) = [0. 0. 1.]  | dot(up) = 1.000
  rot ortho err = 0.000000, det = 1.000000
  z = 0.0000 m
  orientation looks OK for floor tag.
TAG 4
  pos = [0.57794326 0.71598669 0.075     ]
  normal (world) = [-0.18158148 -0.98301313  0.02670862]  | dot(up) = 0.027
  rot ortho err = 0.000000, det = 1.000000
  z = 0.0750 m
  mobile tag normal dot(up) =  0.026708621578987728
TAG 5
  pos = [1.24896878 0.75470769 0.37318592]
  normal (world) = [0.15750718 0.06527124 0.98535839]  | dot(up) = 0.985
  rot ortho err = 0.000000, det = 1.000000
  z = 0.3732 m
  orientation looks OK for floor tag.

Cameras:
CAM kreo1: pos=[ 0.36685743 -1.21978175  1.89932446], rot_err=0.000000, det=1.000000

--- DONE ---


In [1]:
import zmq, msgpack, msgpack_numpy as m
m.patch()
import open3d as o3d
import numpy as np
import time

# --- Settings ---
PORT = 5557
TAG_RADIUS = 0.0375
COLOR_TAG = np.array([0, 1, 0], float)
COLOR_TAG4 = np.array([1, 0, 0], float)
COLOR_CAM = np.array([0.2, 0.6, 1.0], float)

# --- ZeroMQ subscriber ---
ctx = zmq.Context()
sock = ctx.socket(zmq.SUB)
sock.setsockopt(zmq.SUBSCRIBE, b"")  # subscribe to everything
sock.connect(f"tcp://127.0.0.1:{PORT}")

# --- Open3D Setup ---
vis = o3d.visualization.Visualizer()
vis.create_window("Open3D Map", width=1000, height=700, visible=True)

# Floor grid
def make_floor(size=6.0, step=0.25):
    pts = []
    lines = []
    half = size / 2.0
    n = int(size / step) + 1
    idx = 0
    # horizontal lines
    for i in range(n):
        y = -half + i * step
        pts.append([-half, y, 0])
        pts.append([half, y, 0])
        lines.append([idx, idx + 1])
        idx += 2
    # vertical lines
    for i in range(n):
        x = -half + i * step
        pts.append([x, -half, 0])
        pts.append([x, half, 0])
        lines.append([idx, idx + 1])
        idx += 2
    ls = o3d.geometry.LineSet()
    pts = np.array(pts, float)
    ls.points = o3d.utility.Vector3dVector(pts)
    ls.lines = o3d.utility.Vector2iVector(np.array(lines))
    ls.colors = o3d.utility.Vector3dVector(
        np.tile([0.6,0.6,0.6], (len(lines),1)))
    return ls

floor = make_floor()
vis.add_geometry(floor)

origin = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.2)
vis.add_geometry(origin)

# Storage
cam_geoms = {}
tag_geoms = {}

def make_camera_geom():
    # simple pyramid frustum
    pts = np.array([
        [0,0,0],
        [-0.1, 0.07, 0.3],
        [ 0.1, 0.07, 0.3],
        [ 0.1,-0.07, 0.3],
        [-0.1,-0.07, 0.3]
    ], float)
    lines = [
        [0,1],[0,2],[0,3],[0,4],
        [1,2],[2,3],[3,4],[4,1]
    ]
    ls = o3d.geometry.LineSet()
    ls.points = o3d.utility.Vector3dVector(pts)
    ls.lines = o3d.utility.Vector2iVector(np.array(lines))
    ls.colors = o3d.utility.Vector3dVector(np.tile(COLOR_CAM,(len(lines),1)))
    return ls, pts

# Sphere for tags
def make_tag_sphere(is_mobile=False):
    s = o3d.geometry.TriangleMesh.create_sphere(radius=TAG_RADIUS)
    s.compute_vertex_normals()
    s.paint_uniform_color(COLOR_TAG4 if is_mobile else COLOR_TAG)
    return s

print("[viewer] Running Open3D visualizer...")

while True:
    try:
        msg = sock.recv(zmq.NOBLOCK)
    except zmq.Again:
        time.sleep(0.005)
        vis.poll_events()
        vis.update_renderer()
        continue

    data = msgpack.unpackb(msg, object_hook=m.decode, strict_map_key=False)

    tags = data["tags"]
    cams = data["cameras"]

    # Update cameras
    for cam_name in list(cam_geoms.keys()):
        if cam_name not in cams:
            vis.remove_geometry(cam_geoms[cam_name][0])
            cam_geoms.pop(cam_name)

    for cam_name, T in cams.items():
        T = np.array(T)
        if cam_name not in cam_geoms:
            geom, base = make_camera_geom()
            cam_geoms[cam_name] = (geom, base)
            vis.add_geometry(geom)

        geom, base_pts = cam_geoms[cam_name]

        # transform base points
        pts_world = (T[:3,:3] @ base_pts.T + T[:3,3:4]).T
        geom.points = o3d.utility.Vector3dVector(pts_world)
        vis.update_geometry(geom)

    # Update tags
    for tid_str in list(tag_geoms.keys()):
        tid = int(tid_str)
        if tid not in tags:
            vis.remove_geometry(tag_geoms[tid_str])
            tag_geoms.pop(tid_str)

    for tid, T in tags.items():
        T = np.array(T)
        key = str(tid)
        if key not in tag_geoms:
            s = make_tag_sphere(is_mobile=(tid==4))
            vis.add_geometry(s)
            tag_geoms[key] = s
        s = tag_geoms[key]
        s.translate(np.zeros(3), relative=False)
        s.translate(T[:3,3], relative=False)
        vis.update_geometry(s)

    vis.poll_events()
    vis.update_renderer()


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
[viewer] Running Open3D visualizer...


KeyboardInterrupt: 